In [1]:
import requests
import zipfile

def dame_vista(self, id, nombre_sprint, tp_sprint, ruta_habitos):

    response = requests.get(API_URL+f'sprint_habitos_graf/{id}', stream=True)
    
    if response.status_code == 200:
        with zipfile.ZipFile(response.content) as zip_file:
            zip_file.extractall()
        
        # Crear instancias de Figure y FigureCanvas con las figuras descargadas
        graf1 = plt.imread('figura1.png')
        graf2 = plt.imread('figura2.png')
        
        fig1 = plt.figure()
        fig1.imshow(graf1)
        
        fig2 = plt.figure()
        fig2.imshow(graf2)
        
        # Crear instancias de FigureCanvas con las figuras
        canvas1 = FigureCanvas(fig1)
        canvas2 = FigureCanvas(fig2)

        # Agregar los FigureCanvas a los widgets correspondientes en tu interfaz gráfica
        self.Layout_graf1.addWidget(canvas1)
        self.Layout_graf2.addWidget(canvas2)
    else:
        print("Error al descargar las figuras")
        
dame_vista()

TypeError: dame_vista() missing 5 required positional arguments: 'self', 'id', 'nombre_sprint', 'tp_sprint', and 'ruta_habitos'

In [37]:
from services.habitos import HabitoService
from database.database import Session

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from services.utiles import convertir_horas_wh_a_entero
from matplotlib.figure import Figure


DB = Session()

class DS:
    def __init__(self, db):
        self.db = db
        
    def count_consecutive_yes(self, series):
        """
        Cuenta la cantidad de dias consecutivos de 'Yes' en una serie, es decir,
        cuantos dias seguidos se ha cumplido con el habito.

        Parameters
        ----------
        series: pd.Series
            Serie de valores que se quieren contar, estos pueden ser 'Yes' o 'No'.

        Returns
        -------
        int
            Cantidad de dias consecutivos de 'Yes' en la serie.

        Examples
        --------
        >>> count_consecutive_yes(['Yes', 'Yes', 'No', 'Yes'])
        2
        >>> count_consecutive_yes(['Yes', 'Yes', 'Yes', 'Yes'])
        4
        >>> count_consecutive_yes(['Yes', 'Yes', 'Yes', 'No'])
        3
        """
        max_consecutive_yes = 0
        current_consecutive_yes = 0
        for value in series:
            if value == 1:
                current_consecutive_yes += 1
                max_consecutive_yes = max(
                    max_consecutive_yes, current_consecutive_yes)
            else:
                current_consecutive_yes = 0
        return max_consecutive_yes
    
        
        
        
    def get_stats_habitos(self, sprint_id):
            
        habito = HabitoService(self.db)
        habitos = habito.get_habitos_sprint(sprint_id)
        habitos_dict = [d.__dict__ for d in habitos]
        df = pd.DataFrame(habitos_dict)
        
        me_interesa = df.pivot(index='date', columns='habito', values='realizado').reset_index()
        me_interesa = me_interesa.rename_axis(None, axis=1)

        # Habito mayor&menor frecuencia
        frec_habit = me_interesa.eq(1).sum()
        dict = frec_habit.to_dict()
        habits_maxfrec = [i for i in list(
            dict.keys()) if dict[i] == frec_habit.max()]
        habits_minfrec = [i for i in list(
            dict.keys()) if dict[i] == frec_habit.min()]
        habits_ordenados = frec_habit.sort_values()

        # Frecuencia x dia
        new_name = me_interesa.transpose().reset_index()

        new_name.columns = range(len(new_name.columns))
        frec_dia = new_name.eq(1).sum()

        dict_d = frec_dia.to_dict()
        days_maxfrec = [i for i in list(
            dict_d.keys()) if dict_d[i] == frec_dia.max()]
        days_minfrec = [i for i in list(
            dict_d.keys()) if dict_d[i] == frec_dia.min()]

        # Maximos dias consecutivos
        consecutive_yes_count = me_interesa.apply(self.count_consecutive_yes)

        dict_r = consecutive_yes_count.to_dict()
        habits_maxrach = [i for i in list(
            dict_r.keys()) if dict_r[i] == consecutive_yes_count.max()]

        # Porcentaje de dias cumplidos
        me_interesa.eq(1).sum().sum()
        habits_porcent = me_interesa.eq(1).sum().sum() / me_interesa.size * 100

        habits_ordenados = (
            habits_ordenados.drop(['date', 'Status'])
            .index.tolist()
            )

        return f'Habitos con mayor frecuencia: \n {habits_maxfrec}', f'Habitos con menor frecuencia: \n {habits_minfrec}', f'Dia con mas habitos cumplidos: \n {days_maxfrec}', f'Dia con menor habitos cumplidos \n {days_minfrec}', f'Habitos con mayor racha \n {habits_maxrach}', f'Porcentaje total de cumplimiento: {habits_porcent}', f'{habits_ordenados}', len(habits_ordenados)
print(DS(DB).get_stats_habitos(2))

['Ejercitar Flexibilidad', 'Organizar Local', 'Rutina de cabello', 'Visualizar futuro', '💤 Dormir Temprano', 'Leer PSE', '🧘 Meditation', '✍🏽Planificar', '🍩🙅🏽\u200d♂️Sin azúcar', 'Videoterapia', 'Anotar Gastos/Ingresos ', 'Gestión de Diamantes', '🚿Café Helado', '🤳🏽Socializar por Redes Sociales Digitales', '🔥Wake Up On Fire', '💪🏽Hacer Ejercicio', 'Programar', 'Hacer ejercicios de Mr Mandíbulas', 'Rutina Facial', '💦Tomar agua en el inicio del día', '🛌🏽 Tender Cama al Levantarme ']
("Habitos con mayor frecuencia: \n ['💦Tomar agua en el inicio del día', '🛌🏽 Tender Cama al Levantarme ']", "Habitos con menor frecuencia: \n ['date', 'Ejercitar Flexibilidad', 'Organizar Local', 'Rutina de cabello', 'Status', 'Visualizar futuro', '💤 Dormir Temprano']", 'Dia con mas habitos cumplidos: \n [8]', 'Dia con menor habitos cumplidos \n [0]', "Habitos con mayor racha \n ['💦Tomar agua en el inicio del día', '🛌🏽 Tender Cama al Levantarme ']", 'Porcentaje total de cumplimiento: 35.573122529644266', "['Ejerc

In [22]:
from models.sprint import EntrenamientoEjercicio
from services.utiles import convertir_fecha_a_datetime
import pandas as pd

class HabitoService():
    def __init__(self, db):
        self.db = db
        
        
    def get_datos_habitos(self, ruta):
        
        data_raw = pd.read_csv(ruta)
        
        
        data_raw = data_raw.drop(columns=['Rutina']).set_index('Fecha')

        data = (
            pd.melt(data_raw, var_name='Ronda', value_name='Repeticiones', ignore_index=False)    
            .reset_index()
        )

        
        # data = data.set_index('Ejercicio')
        # data.index.name = 'Ejercicio'
        # data = data.sort_values(by='Created Day')
        # data = data.reset_index()
        # data.rename(columns = {'Created Day': 'Fecha'}, inplace = True)
        
        # data['Fecha'] = data['Fecha'].apply(convertir_fecha_a_datetime)
        
        # habito_list = data['Habito'].tolist()
        # fecha_list = data['Fecha'].tolist()
        # realizado_list = [True if i == 'Yes' else False for i in list(data['Realizado'])]
        
        # data = list(zip(habito_list, fecha_list, realizado_list))
data_raw = pd.read_csv(r'C:\Users\User\Downloads\Datos de Entreno Notion\Ejercicios-Tipos.csv')
        
        
#data_raw = data_raw.set_index('Fecha')

data = (
    pd.melt(data_raw, var_name='Ejercicios',id_vars=['Fecha', 'Ronda', 'Rutina'], value_name='Repeticiones',value_vars= 
            # [
            #     'Pull Ups Std', 'Pull Ups Cerradas Inverso', 'Pull Ups Abiertas', 'Elevaciones en barra( completas )', 'Pull Ups Abiertas Inverso',
            #     'Pull Ups L-Sit Neutro', 'Pull Ups Australianas',
            #     'Elevaciones laterales en barra', 'Flexiones Cerradas',
            #     'Flexiones Declinadas', 'Flexiones Pica','Flexiones Diamante', 'Flexiones Explosivas',
            #     'Flexiones Inclinadas', 'Flexiones Std', 'Flexiones Laterales',
            #     'Fondos en Barra', 'Fondos', 'Pistol Squad Asistida',
            #     'Asalto al Frente', 'Sentadillas', 'Sentadilla en Pared ( segs )'
            # ]
            data_raw.keys().tolist()[1:-1]
            )    
    #.reset_index()
)
data = data.dropna(subset=['Repeticiones'])
data = data.reindex(columns = ['Fecha','Ejercicios', 'Repeticiones', 'Ronda', 'Rutina'])
data = data.sort_values(by=['Fecha','Ronda'])

data

,Fecha,Ejercicios,Repeticiones,Ronda,Rutina
0,21/12/2024,Pull Ups Std,4.0,1,rutina muscle
10,21/12/2024,Pull Ups Cerradas Inverso,3.0,1,rutina muscle
20,21/12/2024,Pull Ups Abiertas,3.0,1,rutina muscle
30,21/12/2024,Elevaciones en barra( completas ),6.0,1,rutina muscle
40,21/12/2024,Pull Ups Abiertas Inverso,3.0,1,rutina muscle
...,...,...,...,...,...
109,26/12/2024,Flexiones Pica,3.0,2,rutina pecho interior
119,26/12/2024,Flexiones Diamante,10.0,2,rutina pecho interior
129,26/12/2024,Flexiones Explosivas,4.0,2,rutina pecho interior
139,26/12/2024,Flexiones Inclinadas,15.0,2,rutina pecho interior


In [23]:
data = (
    pd.melt(data_raw, var_name='Ejercicios',id_vars=['Fecha', 'Ronda', 'Rutina'], value_name='Repeticiones',value_vars= 
            # [
            #     'Pull Ups Std', 'Pull Ups Cerradas Inverso', 'Pull Ups Abiertas', 'Elevaciones en barra( completas )', 'Pull Ups Abiertas Inverso',
            #     'Pull Ups L-Sit Neutro', 'Pull Ups Australianas',
            #     'Elevaciones laterales en barra', 'Flexiones Cerradas',
            #     'Flexiones Declinadas', 'Flexiones Pica','Flexiones Diamante', 'Flexiones Explosivas',
            #     'Flexiones Inclinadas', 'Flexiones Std', 'Flexiones Laterales',
            #     'Fondos en Barra', 'Fondos', 'Pistol Squad Asistida',
            #     'Asalto al Frente', 'Sentadillas', 'Sentadilla en Pared ( segs )'
            # ]
            data_raw.keys().tolist()[1:-1]
            )    
    #.reset_index()
)
data = data.dropna(subset=['Repeticiones'])
data = data.reindex(columns = ['Fecha','Ejercicios', 'Repeticiones', 'Ronda', 'Rutina'])
data = data.sort_values(by=['Fecha','Ronda'])
data_raw

,Fecha,Ronda,Pull Ups Std,Pull Ups Cerradas Inverso,Pull Ups Abiertas,Elevaciones en barra( completas ),Pull Ups Abiertas Inverso,Pull Ups L-Sit Neutro,Pull Ups Australianas,Elevaciones laterales en barra,...,Flexiones Inclinadas,Flexiones Std,Flexiones Laterales,Fondos en Barra,Fondos,Pistol Squad Asistida,Asalto al Frente,Sentadillas,Sentadilla en Pared ( segs ),Rutina
0,21/12/2024,1,4.0,3.0,3.0,6.0,3.0,4.0,5.0,8.0,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,rutina muscle
1,22/12/2024,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,10.0,10.0,30.0,rutina pierna hiedra
2,22/12/2024,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,10.0,10.0,30.0,rutina pierna hiedra
3,23/12/2024,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,rutina pecho exterior
4,23/12/2024,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,rutina pecho exterior
5,23/12/2024,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,rutina pecho exterior
6,24/12/2024,1,5.0,4.0,3.0,8.0,3.0,5.0,5.0,10.0,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,rutina muscle
7,24/12/2024,2,4.0,4.0,2.0,6.0,3.0,4.0,5.0,8.0,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,rutina muscle
8,26/12/2024,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rutina pecho interior
9,26/12/2024,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rutina pecho interior


In [14]:
import pandas as pd
df = pd.DataFrame({
    'nombre': ['Juan', 'Ana'],
    'edad_2020': [25, 30],
    'edad_2021': [26, 31]
})

df_melted = pd.melt(df, 
                    id_vars=['nombre'],
                    value_vars=['edad_2020', 'edad_2021'])
df_melted
df

,nombre,edad_2020,edad_2021
0,Juan,25,26
1,Ana,30,31


In [ ]:
class EntrenamientoEjercicio(Base):
    __tablename__ = 'Ejercicios'

    id = Column(Integer(), primary_key=True)
    fecha = Column(Date, nullable=False)
    ronda = Column(Integer, nullable=False)
    pull_ups_std = Column(Integer)
    pull_ups_cerradas_inverso = Column(Integer)
    pull_ups_abiertas = Column(Integer)
    elevaciones_barra_completas = Column(Integer)
    pull_ups_abiertas_inverso = Column(Integer)
    pull_ups_l_sit_neutro = Column(Integer)
    pull_ups_australianas = Column(Integer)
    elevaciones_laterales_barra = Column(Integer)
    flexiones_cerradas = Column(Integer)
    flexiones_declinadas = Column(Integer)
    flexiones_pica = Column(Integer)
    flexiones_diamante = Column(Integer)
    flexiones_explosivas = Column(Integer)
    flexiones_inclinadas = Column(Integer)
    flexiones_std = Column(Integer)
    flexiones_laterales = Column(Integer)
    fondos_barra = Column(Integer)
    fondos = Column(Integer)
    pistol_squad_asistida = Column(Integer)
    asalto_al_frente = Column(Integer)
    sentadillas = Column(Integer)
    sentadilla_en_pared_segs = Column(Integer)
    rutina = Column(String(100), nullable=False)
    
    rutina_id = Column(Integer, ForeignKey('Entrenamientos.id'))
    entrenamiento = relationship('Entrenamiento', back_populates='ejercicio')